In [1]:
# import lib
import tensorflow as tf
import numpy as np
import sys, os,cv2
from sklearn.utils import shuffle
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle

In [2]:
# read all of the data
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels
    
train_images = read_all_images("../../DataSet/STL10/stl10_binary/train_X.bin") / 255.0
train_labels = read_labels    ("../../DataSet/STL10/stl10_binary/train_Y.bin")
test_images  = read_all_images("../../DataSet/STL10/stl10_binary/test_X.bin")  / 255.0
test_labels  = read_labels    ("../../DataSet/STL10/stl10_binary/test_y.bin")

label_encoder= OneHotEncoder(sparse=False,categories='auto')
train_labels = label_encoder.fit_transform(train_labels.reshape((-1,1)))
test_labels  = label_encoder.fit_transform(test_labels.reshape((-1,1)))

In [3]:
# some basic statistic of train and test image // hyper
print(train_images.max((0,1,2)),train_images.min((0,1,2)),train_images.mean((0,1,2)),train_images.std((0,1,2)) )
print(train_images.shape)
print(train_labels.shape)
print(test_images.max((0,1,2)),test_images.min((0,1,2)),test_images.mean((0,1,2)),test_images.std((0,1,2)) )
print(test_images.shape)
print(test_labels.shape)
num_epoch = 50 ; learning_rate = 0.0008; batch_size = 10

[1. 1. 1.] [0. 0. 0.] [0.44671062 0.43980984 0.40664645] [0.26034098 0.25657727 0.27126738]
(5000, 96, 96, 3)
(5000, 10)
[1. 1. 1.] [0. 0. 0.] [0.44723063 0.43964247 0.40495725] [0.2605645  0.25666146 0.26997382]
(8000, 96, 96, 3)
(8000, 10)


In [366]:
# import layers
def tf_softmax(x): return tf.nn.softmax(x)
def tf_elu(x):     return tf.nn.elu(x)
def d_tf_elu(x):   return tf.cast(tf.greater(x,0),tf.float32)  + (tf_elu(tf.cast(tf.less_equal(x,0),tf.float32) * x) + 1.0)
def tf_relu(x):    return tf.nn.relu(x)
def d_tf_relu(x):  return tf.cast(tf.greater(x,0),tf.float32)
def tf_iden(x): return x

class CNN():

    def __init__(self,k,inc,out, stddev=0.05,which_reg=0,act=tf_elu,d_act=d_tf_elu):
        self.w          = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float32))
        self.moving_w   = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float32))
        self.b          = tf.Variable(tf.zeros(out,dtype=tf.float32))
        self.m,self.v   = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.act,self.d_act = act,d_act
        self.which_reg  = which_reg
        
    def getw(self): return [self.w,self.b]
    def feedforward(self,input,stride=1,padding='VALID'):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding)
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient,stride=1,padding='VALID'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad_b      = tf.reduce_mean(grad_middle,(0,1,2))/batch_size
        grad        = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = tf.shape(self.w),  out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) / batch_size
        grad_pass   = tf.nn.conv2d_backprop_input (input_sizes = tf.shape(self.input),filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)

        if self.which_reg == 0:   grad = grad
        if self.which_reg == 0.5: grad = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)
        if self.which_reg == 1:   grad = grad + lamda * tf.sign(self.w)
        if self.which_reg == 1.5: grad = grad + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w
        if self.which_reg == 2:   grad = grad + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)
        if self.which_reg == 2.5: grad = grad + lamda * 2.0 * self.w
        if self.which_reg == 3:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)
        if self.which_reg == 4:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []
        
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middle = m_hat * learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle  )))
        
        update_w.append(tf.assign( self.mb,self.mb*beta1 + (1-beta1) * (grad_b)   ))
        update_w.append(tf.assign( self.vb,self.vb*beta2 + (1-beta2) * (grad_b ** 2)   ))
        m_hatb = self.mb / (1-beta1) ; v_hatb = self.vb / (1-beta2)
        adam_middleb = m_hatb * learning_rate/(tf.sqrt(v_hatb) + adam_e)
        update_w.append(tf.assign(self.b,tf.subtract(self.b,adam_middleb  )))
        
        return grad_pass,update_w
    
    def updatew(self):
        w,h,inc,outc = self.w.shape
        
        tempw = tf.reshape(self.w,(w*h,inc,outc))
        tempw = tf.transpose(tempw,(1,2,0))
        s,U,V = tf.svd(tempw)
        print(s)
        print(tf.reduce_min(s,1,True))
        scaleds = (s-tf.reduce_min(s,1,True))/(tf.reduce_max(s,1,True)-tf.reduce_min(s,1,True))
        _,n   = s.shape
        n     = n//2
        print(n)
        neww  = U[:,:,:n] @ tf.matrix_diag(scaleds)[:,:n,:] 
        neww  = tf.transpose(neww,(2,0,1))
        neww  = tf.reshape(neww,(w,h,inc,outc))
        
        update = []
        update.append(tf.assign(self.moving_w,self.moving_w * 0.9 + 0.1 * neww))
        update.append(tf.assign(self.w,self.moving_w))
        return update

In [367]:
# create the svd layer
class svd_layer():
    
    def __init__(self,batch,size,width):
        self.n = size
        self.moving_s = tf.Variable(tf.zeros((batch_size,size),dtype=tf.float32))
        self.moving_u = tf.Variable(tf.zeros((batch_size,width**2,size),dtype=tf.float32))
        self.moving_v = tf.Variable(tf.zeros((batch_size,size,size),dtype=tf.float32))
    
    def feedforward(self,data,training_phase):
        
        with tf.device('/cpu:0'):
            s,U,V = tf.svd(data)
        print(s)
        print(U)
        print(V)
        smin = tf.reduce_min(s,1,keepdims=True)
        smax = tf.reduce_max(s,1,keepdims=True)
        scaleds = (s - smin)/(smax-smin + 1e-8)
        def training_fn():
            data      = U[:,:,:] @ tf.matrix_diag(s)[:,:,:] @ tf.transpose(V,(0,2,1))[:,:,:]
            data = data  * tf.reduce_mean(tf.transpose(tf.abs(V),(0,2,1)) * scaleds[:,:,None] ,(1),keepdims=True)
            # data      = data * tf.reduce_mean(tf.transpose(tf.abs(V),(0,2,1)) * scaleds[:,:,None] ,(1),keepdims=True)
            update = []
            update.append(tf.assign(self.moving_u,self.moving_u*0.9 + 0.1 * U))
            update.append(tf.assign(self.moving_v,self.moving_v*0.9 + 0.1 * V))
            return data,update
            
        def testing_fn():
            data      = U[:,:,:] @ tf.matrix_diag(s)[:,:,:] @ tf.transpose(V,(0,2,1))[:,:,:]
            # data      = data * tf.reduce_mean(tf.transpose(tf.abs(V),(0,2,1)) * scaleds[:,:,None] ,(1),keepdims=True)
            update = []
            update.append(tf.assign(self.moving_u,self.moving_u))
            update.append(tf.assign(self.moving_v,self.moving_v))
            return data,update
        
        data,update  = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return data,update

In [368]:
# restart the graph 
# sess.close()
# tf.reset_default_graph()
learning_rate = 0.0008; batch_size = 100

l1 = CNN(3,3, 16)
l2 = CNN(3,16,16)
l3 = CNN(3,16,16)
l3_svd = svd_layer(100,16,24)

l4 = CNN(3,16,32)
l5 = CNN(3,32,32)
l6 = CNN(3,32,32)
l6_svd = svd_layer(100,32,6)

l7 = CNN(3,32,64)
l8 = CNN(3,64,64)
l9 = CNN(3,64,10)

In [369]:
# build graph 
x = tf.placeholder(shape=(batch_size,96,96,3),dtype=tf.float32)
y = tf.placeholder(shape=(batch_size,10),dtype=tf.float32)
is_train = tf.placeholder_with_default(False,())

layer1 = l1.feedforward(x       ,padding='SAME',stride=2)
layer2 = l2.feedforward(layer1  ,padding='SAME',stride=2) 
layer3 = l3.feedforward(layer2  ,padding='SAME',stride=1)

layer4 = l4.feedforward(layer3  ,padding='SAME',stride=2) 
layer5 = l5.feedforward(layer4  ,padding='SAME',stride=2)
layer6 = l6.feedforward(layer5  ,padding='SAME',stride=1)

layer7 = l7.feedforward(layer6  ,padding='SAME',stride=1)
layer8 = l8.feedforward(layer7  ,padding='SAME',stride=1) 
layer9 = l9.feedforward(layer8  ,padding='SAME',stride=1)

all_update = l1.updatew() + l2.updatew() + l3.updatew() + \
             l4.updatew() + l5.updatew() + l6.updatew() + \
             l7.updatew() + l8.updatew() + l9.updatew() 

final_layer = tf.reduce_mean(layer9,axis=(1,2))
cost        = tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_layer,labels=y)
auto_train  = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

final_softmax      = tf_softmax(final_layer)
correct_prediction = tf.equal(tf.argmax(final_softmax, 1), tf.argmax(y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Tensor("Svd_383:0", shape=(3, 9), dtype=float32)
Tensor("Min_391:0", shape=(3, 1), dtype=float32)
4
Tensor("Svd_384:0", shape=(16, 9), dtype=float32)
Tensor("Min_394:0", shape=(16, 1), dtype=float32)
4
Tensor("Svd_385:0", shape=(16, 9), dtype=float32)
Tensor("Min_397:0", shape=(16, 1), dtype=float32)
4
Tensor("Svd_386:0", shape=(16, 9), dtype=float32)
Tensor("Min_400:0", shape=(16, 1), dtype=float32)
4
Tensor("Svd_387:0", shape=(32, 9), dtype=float32)
Tensor("Min_403:0", shape=(32, 1), dtype=float32)
4
Tensor("Svd_388:0", shape=(32, 9), dtype=float32)
Tensor("Min_406:0", shape=(32, 1), dtype=float32)
4
Tensor("Svd_389:0", shape=(32, 9), dtype=float32)
Tensor("Min_409:0", shape=(32, 1), dtype=float32)
4
Tensor("Svd_390:0", shape=(64, 9), dtype=float32)
Tensor("Min_412:0", shape=(64, 1), dtype=float32)
4
Tensor("Svd_391:0", shape=(64, 9), dtype=float32)
Tensor("Min_415:0", shape=(64, 1), dtype=float32)
4


In [ ]:
# start the training 
# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())
num_epoch = 30000 ; avg_acc_train = 0; avg_acc_test  = 0; train_more = 1

for iter in range(num_epoch):
    
    test_images,test_labels   = shuffle(test_images,test_labels);

    for i in range(train_more):
        train_images,train_labels = shuffle(train_images,train_labels);
        for current_batch_index in range(0,len(train_images),batch_size):
            current_data  = train_images[current_batch_index:current_batch_index+batch_size].astype(np.float32)
            current_label = train_labels[current_batch_index:current_batch_index+batch_size].astype(np.float32)
            sess_results  = sess.run([accuracy,auto_train],feed_dict={x:current_data,y:current_label})
            sys.stdout.write(str(i) + ' Current Iter : ' + str(iter) + '/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(train_images)) + ' acc : ' + str(sess_results[0]) + '\r')
            sys.stdout.flush(); avg_acc_train = avg_acc_train + sess_results[0]
        
    for current_batch_index in range(0,len(test_images), batch_size):
        current_data  = test_images[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        current_label = test_labels[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        sess_results  = sess.run([accuracy],feed_dict={x:current_data,y:current_label})
        sys.stdout.write('Current Iter : ' + str(iter) + '/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(test_images)) + ' acc : ' + str(sess_results[0]) + '\r')
        sys.stdout.flush(); avg_acc_test = avg_acc_test + sess_results[0]        
        

    if avg_acc_train/(len(train_images)/batch_size)/train_more > 0.9 and avg_acc_test/(len(test_images)/batch_size) < 0.5 :  
        print('\n-----------reset')
        sess.run(all_update)
    if avg_acc_test/(len(test_images)/batch_size) > 0.5:
        print('-----------------------------------------')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
        print('-----------------------------------------')

    print("Current : "+ str(iter) + " Acc : " + str(avg_acc_train/(len(train_images)/batch_size)/train_more) + " Test Acc : " + str(avg_acc_test/(len(test_images)/batch_size)) + '\n')
    avg_acc_train = 0 ; avg_acc_test  = 0

Current : 0 Acc : 0.5501999992132187 Test Acc : 0.4532499972730875

Current : 1 Acc : 0.5513999980688095 Test Acc : 0.46212499812245367

Current : 2 Acc : 0.5718000024557114 Test Acc : 0.4528749980032444

Current : 3 Acc : 0.5822000032663346 Test Acc : 0.46037499867379666

Current : 4 Acc : 0.5914000022411346 Test Acc : 0.46399999782443047

Current : 5 Acc : 0.6077999985218048 Test Acc : 0.46312499791383743

Current : 6 Acc : 0.6226000010967254 Test Acc : 0.4641249947249889

Current : 7 Acc : 0.6413999998569488 Test Acc : 0.46674999594688416

Current : 8 Acc : 0.6484000027179718 Test Acc : 0.4563749972730875

Current : 9 Acc : 0.6692000043392181 Test Acc : 0.44937499761581423

Current : 10 Acc : 0.6896000027656555 Test Acc : 0.47649999558925626

Current : 11 Acc : 0.7181999969482422 Test Acc : 0.46312499307096006

Current : 12 Acc : 0.7177999997138977 Test Acc : 0.46712499521672723

Current : 13 Acc : 0.7619999992847443 Test Acc : 0.4692499987781048

Current : 14 Acc : 0.75619999885559

Current : 115 Acc : 0.6121999979019165 Test Acc : 0.42562499903142453

Current : 116 Acc : 0.6320000004768371 Test Acc : 0.41362499855458734

Current : 117 Acc : 0.6410000014305115 Test Acc : 0.4304999988526106

Current : 118 Acc : 0.663800002336502 Test Acc : 0.41474999748170377

Current : 119 Acc : 0.6835999989509582 Test Acc : 0.4454999964684248

Current : 120 Acc : 0.7332000029087067 Test Acc : 0.41862499825656413

Current : 121 Acc : 0.7505999994277954 Test Acc : 0.4366249989718199

Current : 122 Acc : 0.7551999998092651 Test Acc : 0.41849999837577345

Current : 123 Acc : 0.7943999969959259 Test Acc : 0.4379999995231628

Current : 124 Acc : 0.8500000023841858 Test Acc : 0.43212499767541884

Current : 125 Acc : 0.8378000009059906 Test Acc : 0.4102499973028898

Current : 126 Acc : 0.8470000004768372 Test Acc : 0.4127499971538782

Current : 127 Acc : 0.8855999982357026 Test Acc : 0.40949999913573265

Current Iter : 128/30000 batch : 7900/8000 acc : 0.3793
-----------reset
Current : 1

Current : 229 Acc : 0.8966000008583069 Test Acc : 0.40999999903142453

Current : 230 Acc : 0.885600004196167 Test Acc : 0.42949999757111074

Current Iter : 231/30000 batch : 7900/8000 acc : 0.4694
-----------reset
Current : 231 Acc : 0.9107999992370606 Test Acc : 0.42812499813735483

Current : 232 Acc : 0.10000000029802322 Test Acc : 0.10000000009313226

Current : 233 Acc : 0.10000000018626451 Test Acc : 0.10012499988079071

Current : 234 Acc : 0.10000000022351742 Test Acc : 0.10012499955482781

Current : 235 Acc : 0.1001999994367361 Test Acc : 0.10000000023283065

Current : 236 Acc : 0.11019999995827674 Test Acc : 0.102125000115484

Current : 237 Acc : 0.0997999995201826 Test Acc : 0.10000000027939678

Current : 238 Acc : 0.10000000029802322 Test Acc : 0.09999999986030161

Current : 239 Acc : 0.1 Test Acc : 0.100124999554827815

Current : 240 Acc : 0.10459999971091748 Test Acc : 0.14387500043958426

Current : 241 Acc : 0.16019999966025353 Test Acc : 0.16962499963119626

Current : 242 

Current : 342 Acc : 0.10000000029802322 Test Acc : 0.10012500016018748

Current : 343 Acc : 0.1303999999165535 Test Acc : 0.1706250005401671

Current : 344 Acc : 0.16100000068545342 Test Acc : 0.17724999990314244

Current : 345 Acc : 0.1844000007212162 Test Acc : 0.19637500001117586

Current : 346 Acc : 0.2029999989271164 Test Acc : 0.19662500014528633

Current : 347 Acc : 0.2183999991416931 Test Acc : 0.21500000078231096

Current : 348 Acc : 0.23599999994039536 Test Acc : 0.2508750006556511

Current : 349 Acc : 0.256199999153614 Test Acc : 0.2685000000521541

Current : 350 Acc : 0.2610000005364418 Test Acc : 0.24587499927729367

Current : 351 Acc : 0.2846000024676323 Test Acc : 0.2951250024139881

Current : 352 Acc : 0.3082000005245209 Test Acc : 0.30275000240653754

Current : 353 Acc : 0.3152000021934509 Test Acc : 0.32700000181794164

Current : 354 Acc : 0.34480000138282774 Test Acc : 0.3337500009685755

Current : 355 Acc : 0.3526000016927719 Test Acc : 0.3580000001937151

Current :

Current : 455 Acc : 0.29140000134706495 Test Acc : 0.3247500009834766

Current : 456 Acc : 0.321400004029274 Test Acc : 0.340875000320375

Current : 457 Acc : 0.337399999499321 Test Acc : 0.34825000185519456

Current : 458 Acc : 0.33940000116825103 Test Acc : 0.35075000040233134

Current : 459 Acc : 0.361599999666214 Test Acc : 0.3653749987483025

Current : 460 Acc : 0.38599999964237214 Test Acc : 0.3765000008046627

Current : 461 Acc : 0.382800001502037 Test Acc : 0.38812500052154064

Current : 462 Acc : 0.3946000009775162 Test Acc : 0.36612500064074993

Current : 463 Acc : 0.40900000035762785 Test Acc : 0.3996250007301569

Current : 464 Acc : 0.4145999979972839 Test Acc : 0.3972499992698431

Current : 465 Acc : 0.44179999887943266 Test Acc : 0.4274999961256981

Current : 466 Acc : 0.44999999821186065 Test Acc : 0.43099999949336054

Current : 467 Acc : 0.47459999442100526 Test Acc : 0.4306249964982271

Current : 468 Acc : 0.4817999958992004 Test Acc : 0.43312499634921553

Current : 46

Current : 569 Acc : 0.3385999998450279 Test Acc : 0.3273750027641654

Current : 570 Acc : 0.35400000274181365 Test Acc : 0.3430000016465783

Current : 571 Acc : 0.36220000267028807 Test Acc : 0.36075000148266556

Current : 572 Acc : 0.3889999985694885 Test Acc : 0.375

Current : 573 Acc : 0.386599999666214 Test Acc : 0.3886250004172325

Current : 574 Acc : 0.40439999997615816 Test Acc : 0.3982499971985817

Current : 575 Acc : 0.41059999763965604 Test Acc : 0.41062499955296516

Current : 576 Acc : 0.4247999972105026 Test Acc : 0.4063749980181456

Current : 577 Acc : 0.43979999601840974 Test Acc : 0.379874999076128

Current : 578 Acc : 0.4443999987840652 Test Acc : 0.41749999634921553

Current : 579 Acc : 0.4613999962806702 Test Acc : 0.42287499718368055

Current : 580 Acc : 0.46999999523162844 Test Acc : 0.43949999548494817

Current : 581 Acc : 0.4829999947547913 Test Acc : 0.44799999669194224

Current : 582 Acc : 0.49819999396800996 Test Acc : 0.4504999969154596

Current : 583 Acc : 0.

Current : 683 Acc : 0.36039999961853025 Test Acc : 0.35450000055134295

Current : 684 Acc : 0.37080000042915345 Test Acc : 0.35962499752640725

Current : 685 Acc : 0.3721999990940094 Test Acc : 0.3755000002682209

Current : 686 Acc : 0.39279999732971194 Test Acc : 0.37862499989569187

Current : 687 Acc : 0.3983999991416931 Test Acc : 0.38912499845027926

Current : 688 Acc : 0.4015999990701675 Test Acc : 0.37749999985098837

Current : 689 Acc : 0.412399999499321 Test Acc : 0.4062499973922968

Current : 690 Acc : 0.4267999956011772 Test Acc : 0.3889999993145466

Current : 691 Acc : 0.4311999976634979 Test Acc : 0.4131249986588955

Current : 692 Acc : 0.44759999692440033 Test Acc : 0.39337499924004077

Current : 693 Acc : 0.4557999974489212 Test Acc : 0.4267499975860119

Current : 694 Acc : 0.47599999845027924 Test Acc : 0.45049999840557575

Current : 695 Acc : 0.47979999721050265 Test Acc : 0.43124999925494195

Current : 696 Acc : 0.49620000064373015 Test Acc : 0.44874999709427354

Curre

Current : 796 Acc : 0.3138000002503395 Test Acc : 0.330875001475215

Current : 797 Acc : 0.32820000022649765 Test Acc : 0.3441250005736947

Current : 798 Acc : 0.3411999985575676 Test Acc : 0.3245000006631017

Current : 799 Acc : 0.35119999945163727 Test Acc : 0.35775000024586917

Current : 800 Acc : 0.358600001335144 Test Acc : 0.3701249998062849

Current : 801 Acc : 0.3729999995231628 Test Acc : 0.37724999748170374

Current : 802 Acc : 0.37479999750852583 Test Acc : 0.37949999831616876

Current : 803 Acc : 0.3979999989271164 Test Acc : 0.3921249993145466

Current : 804 Acc : 0.40680000066757205 Test Acc : 0.39437499828636646

Current : 805 Acc : 0.41499999701976775 Test Acc : 0.4147499978542328

Current : 806 Acc : 0.4332000005245209 Test Acc : 0.4079999983310699

Current : 807 Acc : 0.4383999979496002 Test Acc : 0.4034999988973141

Current : 808 Acc : 0.4537999951839447 Test Acc : 0.41924999803304674

Current : 809 Acc : 0.45919999837875364 Test Acc : 0.43124999701976774

Current : 

# Reference 
1. Brownlee, J. (2017). How to One Hot Encode Sequence Data in Python. Machine Learning Mastery. Retrieved 9 December 2018, from https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
2. tf.placeholder_with_default | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/placeholder_with_default
3. tf.nn.softmax_cross_entropy_with_logits | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits
4. line, O. (2018). Output without new line. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/2623470/output-without-new-line
5. shell?, H. (2018). How to tell if tensorflow is using gpu acceleration from inside python shell?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/38009682/how-to-tell-if-tensorflow-is-using-gpu-acceleration-from-inside-python-shell
6. GPU?, H. (2018). How to use TensorFlow GPU?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/51306862/how-to-use-tensorflow-gpu
7. Batch normalization: theory and how to use it with Tensorflow. (2018). Towards Data Science. Retrieved 9 December 2018, from https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
8. Batch normalization: theory and how to use it with Tensorflow. (2018). Towards Data Science. Retrieved 9 December 2018, from https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
9. tf.reset_default_graph | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/reset_default_graph
10. tf.Session | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/Session
11. tf.nn.moments | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/nn/moments
12. CMD?, H. (2018). How do I run two commands in one line in Windows CMD?. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/8055371/how-do-i-run-two-commands-in-one-line-in-windows-cmd
13. loop, B. (2018). Batch script loop. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/2591758/batch-script-loop
14. tf.train.MomentumOptimizer | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/train/MomentumOptimizer
15. Test if two numpy arrays are (close to) equal, i. (2018). Test if two numpy arrays are (close to) equal, including shape. Stack Overflow. Retrieved 9 December 2018, from https://stackoverflow.com/questions/32874840/test-if-two-numpy-arrays-are-close-to-equal-including-shape
16. tf.linalg.diag | TensorFlow. (2018). TensorFlow. Retrieved 9 December 2018, from https://www.tensorflow.org/api_docs/python/tf/linalg/diag
17. tf.layers.batch_normalization | TensorFlow. (2018). TensorFlow. Retrieved 10 December 2018, from https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization
18. tf.layers.batch_normalization | TensorFlow. (2018). TensorFlow. Retrieved 10 December 2018, from https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization
19. error, t. (2018). tf.layers.batch_normalization large test error. Stack Overflow. Retrieved 10 December 2018, from https://stackoverflow.com/questions/43234667/tf-layers-batch-normalization-large-test-error
20. 
